<a href="https://colab.research.google.com/github/Burbina200/Proyecto-IMT2118/blob/main/proyecto_geoespaciales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Geo Espaciales

## Integrantes:
- Camila Basulto
- Benjamín Urbina

## Tema:
- Análisis de cobertura de servicios existentes
- Identificación de zonas aptas para construir

Haremos un mapa con zonas aptas para urbanizar, con recomendaciones específicas sobre qué tipo de infraestructura construir en cada una.


# Carga de datos raw

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import geemap,ee

In [2]:
bomberos_df = gpd.read_file('data/bomberos/layer_cuerpos_de_bomberos_20231110074853.shp')
carabineros_df = gpd.read_file('data/carabineros/layer_cuarteles_20220309024623.shp')
salud_df = gpd.read_file('data/salud/layer_establecimientos_de_salud_20240712091725.shp')
educacion_df = gpd.read_file('data/educacion/layer_establecimientos_educacion_escolar_20220309024120.shp')
parvularia_df = gpd.read_file('data/educacion/layer_establecimientos_educacion_parvularia_20220309024143.shp')

In [3]:
carabineros_df.to_crs(bomberos_df.crs)
salud_df.to_crs(bomberos_df.crs)
educacion_df.to_crs(bomberos_df.crs)
parvularia_df.to_crs(bomberos_df.crs)

,AGNO,ID_ESTAB,ORIGEN,NOM_ESTAB,COD_REG_ES,COD_PRO_ES,COD_COM_ES,NOM_COM_ES,COD_DEPROV,NOM_DEPROV,...,NT2_SI,NT2_H,NT2_M,N_TOTAL,DIRECCION,NUMERO,REFERENCIA,LATITUD,LONGITUD,geometry
0,2021.0,2.0,1.0,PARVULARIO LAS ESPIGUITAS,15.0,151.0,15101.0,ARICA,151.0,ARICA,...,0.0,41.0,43.0,170.0,RONALD ROSS,2165.0,POB. JUAN NOE,-18.469862,-70.302757,POINT (-70.30276 -18.46986)
1,2021.0,8.0,1.0,COLEGIO INTEGRADO EDUARDO FREI MONTALVA,15.0,151.0,15101.0,ARICA,151.0,ARICA,...,0.0,23.0,9.0,60.0,SAN MARCOS,611.0,ESQUINA GENERAL LAGOS,-18.482491,-70.317581,POINT (-70.31758 -18.48249)
2,2021.0,9.0,1.0,ESCUELA REPUBLICA DE ISRAEL,15.0,151.0,15101.0,ARICA,151.0,ARICA,...,0.0,52.0,55.0,190.0,MAGALLANES,1860.0,ENTRE SIMÓN BOLÍVAR Y BORGOÑO POBLACIÓN MAGIS...,-18.486691,-70.304520,POINT (-70.30452 -18.48669)
3,2021.0,10.0,1.0,ESCUELA REPUBLICA DE FRANCIA,15.0,151.0,15101.0,ARICA,151.0,ARICA,...,0.0,4.0,3.0,15.0,MARIA ILIA DEL PINO,728.0,INTERSECCIÓN DE CALLE SANTA MARÍA CON MARÍA IL...,-18.474369,-70.309036,POINT (-70.30904 -18.47437)
4,2021.0,11.0,1.0,ESC. GRAL. PEDRO LAGOS MARCHANT,15.0,151.0,15101.0,ARICA,151.0,ARICA,...,0.0,8.0,9.0,31.0,AVENIDA R SOTOMAYOR,1639.0,EN EL CERRO LA CRUZ,-18.491011,-70.311577,POINT (-70.31158 -18.49101)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11946,2021.0,171806.0,3.0,MINAS DEL PRADO,16.0,163.0,16302.0,COIHUECO,0.0,None,...,0.0,0.0,0.0,21.0,SECTOR MINAS DEL PRADO,0.0,S/N,-36.667357,-71.706761,POINT (-71.70676 -36.66736)
11947,2021.0,171901.0,3.0,EL CAMARONCITO,16.0,163.0,16303.0,ÑIQUÉN,0.0,None,...,0.0,0.0,0.0,28.0,DARÍO SALAS,0.0,SECTOR SAN GREGORIO S/N,-36.282696,-71.817511,POINT (-71.81751 -36.2827)
11948,2021.0,172001.0,3.0,PULGARCITO,16.0,163.0,16304.0,SAN FABIÁN,0.0,None,...,0.0,2.0,3.0,33.0,18 DE SEPTIEMBRE,0.0,S/N,-36.557380,-71.548711,POINT (-71.54871 -36.55738)
11949,2021.0,172101.0,3.0,MANZANITO,16.0,163.0,16305.0,SAN NICOLÁS,0.0,None,...,0.0,0.0,0.0,46.0,PANAMERICANA NORTE,0.0,S/N,-36.539583,-72.085723,POINT (-72.08572 -36.53958)


# Preprocesamiento de datos

In [4]:
#Se consideran las siguientes ciudades para el análisis:
# ARICA, LA SERENA, VIÑA DEL MAR, VALPARAÍSO, CONCEPCIÓN, RANCAGUA, PUERTO MONTT, VALDIVIA
#Se incluyen las variantes de nombres para evitar problemas con los registros distintos entre los datasets.
ciudades_list = ['ARICA', 'LA SERENA', 'VIÑA DEL MAR', 'VINA DEL MAR','VALPARAÍSO', 'VALPARAISO', 'CONCEPCIÓN', 'CONCEPCION', 'RANCAGUA','PUERTO MONTT', 'VALDIVIA']

### Cuerpos de Bomberos

In [5]:
bomberos_df.columns

Index(['OBJECTID', 'NOMBRE_DEL', 'TIPO_DE_VL', 'NUMERACIó', 'NOMBRE_DE',
       'COMUNA', 'PROVINCIA', 'REGIóN', 'FID_CL_COM', 'ID_2002', 'NOMBRE',
       'FID_CL_REG', 'ID_2002_20', 'NOMBRE_1', 'geometry'],
      dtype='object')

In [6]:
bomberos_df.head()

,OBJECTID,NOMBRE_DEL,TIPO_DE_VL,NUMERACIó,NOMBRE_DE,COMUNA,PROVINCIA,REGIóN,FID_CL_COM,ID_2002,NOMBRE,FID_CL_REG,ID_2002_20,NOMBRE_1,geometry
0,308.0,Puerto Williams,Calle,220,O Higgins,Cabo de Hornos (Ex Navarino),Antartica Chilena,Región de Magallanes y la Antártica Chilena,332.0,12201,Cabo de Hornos,14.0,12,Magallanes y de la Antartica Chileno,POINT (-67.60351 -54.93455)
1,306.0,Puerto Porvenir,Calle,80,Bernardo Phillipi,Porvenir,Tierra del Fuego,Región de Magallanes y la Antártica Chilena,331.0,12301,Porvenir,14.0,12,Magallanes y de la Antartica Chileno,POINT (-70.36875 -53.29513)
2,305.0,Punta Arenas,None,824,Chiloé,Punta Arenas,Magallanes,Región de Magallanes y la Antártica Chilena,340.0,12101,Punta Arenas,14.0,12,Magallanes y de la Antartica Chileno,POINT (-70.91018 -53.16251)
3,307.0,Puerto Natales,Calle,1455,Simón Bolivar,Natales,Última Esperanza,Región de Magallanes y la Antártica Chilena,333.0,12401,Puerto Natales,14.0,12,Magallanes y de la Antartica Chileno,POINT (-72.48352 -51.73077)
4,303.0,Cochrane,Avenida,360,Bernardo O Higgins,Cochrane,Capitan Prat,Región de Aysén del General Carlos Ibañez del ...,336.0,11301,Cochrane,13.0,11,Aisen del Gral. Carlos IbaOez del Campo,POINT (-72.57934 -47.25521)


In [7]:
print(f"Bomberos DataFrame: {bomberos_df.shape[0]} registros" ) 

Bomberos DataFrame: 309 registros


In [8]:
bomberos = bomberos_df[['NOMBRE_DEL','COMUNA', 'geometry']]
bomberos['COMUNA'] = bomberos['COMUNA'].str.upper()
bomberos = bomberos[bomberos['COMUNA'].isin(ciudades_list)] #Selecciona las comunas de interés
bomberos.reset_index(drop=True, inplace=True)
bomberos.rename(columns={'NOMBRE_DEL': 'NOMBRE_ESTABLECIMIENTO'}, inplace=True)
bomberos

c:\Users\basul\OneDrive\Desktop\u\geoespaciales\geospatial\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,NOMBRE_ESTABLECIMIENTO,COMUNA,geometry
0,Puerto Montt,PUERTO MONTT,POINT (-72.93722 -41.47155)
1,Carretera Austral,PUERTO MONTT,POINT (-72.84177 -41.47161)
2,Valdivia,VALDIVIA,POINT (-73.24511 -39.81223)
3,Huellelhue,VALDIVIA,POINT (-73.11731 -39.7921)
4,Concepción,CONCEPCIÓN,POINT (-73.04221 -36.82809)
5,Rancagua,RANCAGUA,POINT (-70.74684 -34.16951)
6,Valparaíso,VALPARAÍSO,POINT (-71.62887 -33.03889)
7,Viña del Mar,VIÑA DEL MAR,POINT (-71.5399 -33.01762)
8,La Serena,LA SERENA,POINT (-71.24998 -29.90605)
9,Arica,ARICA,POINT (-70.31145 -18.48079)


In [9]:
bomberos.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   NOMBRE_ESTABLECIMIENTO  10 non-null     object  
 1   COMUNA                  10 non-null     object  
 2   geometry                10 non-null     geometry
dtypes: geometry(1), object(2)
memory usage: 372.0+ bytes


In [10]:
print(f"Bomberos DataFrame filtrado: {bomberos.shape[0]} registros" )

Bomberos DataFrame filtrado: 10 registros


In [11]:
bomberos['COMUNA'].unique() #Comprobamos que están las comunas que nos interesan

array(['PUERTO MONTT', 'VALDIVIA', 'CONCEPCIÓN', 'RANCAGUA', 'VALPARAÍSO',
       'VIÑA DEL MAR', 'LA SERENA', 'ARICA'], dtype=object)

### Cuarteles de Carabineros

In [12]:
carabineros_df.columns

Index(['NOMBRE_REG', 'NOMBRE_PRO', 'NOMBRE_COM', 'NOMBRE_UNI', 'TIPO_DE_UN',
       'TIPO_DE_VI', 'NOMBRE_DE', 'NUMERO', 'POINT_X', 'POINT_Y', 'HECHO',
       'RG', 'PREFECTURA', 'TIPO', 'ZONA', 'UNI_COD', 'UNID_ENCRI',
       'geometry'],
      dtype='object')

In [13]:
carabineros_df.head()

,NOMBRE_REG,NOMBRE_PRO,NOMBRE_COM,NOMBRE_UNI,TIPO_DE_UN,TIPO_DE_VI,NOMBRE_DE,NUMERO,POINT_X,POINT_Y,HECHO,RG,PREFECTURA,TIPO,ZONA,UNI_COD,UNID_ENCRI,geometry
0,REGION ARICA Y PARINACOTA,ARICA,ARICA,1RA. COM. ARICA,COMISARIA,AVENIDA,SANTA MARIA,900,-70.311605,-18.478186,2014,15.0,ARICA,None,ARICA Y PARINACOTA,15001,4e007229b70f090dd4f3da67df0dad03,POINT Z (-70.3116 -18.47819 0)
1,REGION ARICA Y PARINACOTA,PARINACOTA,PUTRE,2DA. COM. PUTRE,COMISARIA,CALLE,RIQUELME,298,-69.559351,-18.195069,2014,15.0,ARICA,None,ARICA Y PARINACOTA,15002,19b6f5e29f49c81be5ee2950a58c8ede,POINT Z (-69.55935 -18.19507 0)
2,REGION ARICA Y PARINACOTA,ARICA,ARICA,3RA. COM. ARICA,COMISARIA,AVENIDA,AVENIDA LOA,1475,-70.293810,-18.470430,2014,15.0,ARICA,None,ARICA Y PARINACOTA,15003,bf93f1772d2469dbd77638b6ce784178,POINT Z (-70.29381 -18.47043 0)
3,REGION ARICA Y PARINACOTA,PARINACOTA,GENERAL LAGOS,RETEN ALCERRECA (F),RETEN,RUTA,RUTA A-23,KM 40,-69.660406,-17.992306,2014,15.0,ARICA,None,ARICA Y PARINACOTA,15004,None,POINT Z (-69.66041 -17.99231 0)
4,REGION ARICA Y PARINACOTA,PARINACOTA,PUTRE,RETEN CAQUENA (F),RETEN,RUTA,RUTA A-159,KM 12,-69.201796,-18.054583,2014,15.0,ARICA,None,ARICA Y PARINACOTA,15005,None,POINT Z (-69.2018 -18.05458 0)


In [14]:
print(f"Carabineros DataFrame: {carabineros_df.shape[0]} registros" ) 

Carabineros DataFrame: 874 registros


In [15]:
carabineros = carabineros_df[['NOMBRE_UNI','NOMBRE_COM', 'geometry']]
carabineros['NOMBRE_COM'] = carabineros['NOMBRE_COM'].str.upper()
carabineros = carabineros[carabineros['NOMBRE_COM'].isin(ciudades_list)] #Selecciona las comunas de interés
carabineros.reset_index(drop=True, inplace=True)
carabineros.rename(columns={'NOMBRE_UNI': 'NOMBRE_ESTABLECIMIENTO', 'NOMBRE_COM': 'COMUNA'}, inplace=True)
carabineros

c:\Users\basul\OneDrive\Desktop\u\geoespaciales\geospatial\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,NOMBRE_ESTABLECIMIENTO,COMUNA,geometry
0,1RA. COM. ARICA,ARICA,POINT Z (-70.3116 -18.47819 0)
1,3RA. COM. ARICA,ARICA,POINT Z (-70.29381 -18.47043 0)
2,RETEN CENTRAL (F),ARICA,POINT Z (-69.95928 -18.36165 0)
3,RETEN POCONCHILE (M.C.),ARICA,POINT Z (-70.06675 -18.45249 0)
4,RETEN SAN MIGUEL DE AZAPA,ARICA,POINT Z (-70.17958 -18.51891 0)
5,4TA. COM. CHACALLUTA (F),ARICA,POINT Z (-70.30444 -18.36167 0)
6,SCOM. CHINCHORRO NORTE,ARICA,POINT Z (-70.29279 -18.44581 0)
7,RETEN NIEBLA,VALDIVIA,POINT Z (-73.3944 -39.8651 0)
8,TCIA. SUBTTE. RUBEN MOGOLLONES A.,VALDIVIA,POINT Z (-73.20145 -39.83304 0)
9,1RA. COM. VALDIVIA,VALDIVIA,POINT Z (-73.23519 -39.81787 0)


In [16]:
carabineros.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   NOMBRE_ESTABLECIMIENTO  55 non-null     object  
 1   COMUNA                  55 non-null     object  
 2   geometry                55 non-null     geometry
dtypes: geometry(1), object(2)
memory usage: 1.4+ KB


In [17]:
print(f"Carabineros DataFrame filtrado: {carabineros.shape[0]} registros" )

Carabineros DataFrame filtrado: 55 registros


In [18]:
carabineros['COMUNA'].unique() #Comprobamos que están las comunas que nos interesan

array(['ARICA', 'VALDIVIA', 'PUERTO MONTT', 'CONCEPCION', 'RANCAGUA',
       'VALPARAISO', 'VINA DEL MAR', 'LA SERENA'], dtype=object)

### Establecimientos de salud

In [19]:
salud_df.columns

Index(['COD_ANT', 'COD_VIG', 'COD_M_ANT', 'COD_M_NUEV', 'COD_REG', 'NOM_REG',
       'COD_DEP', 'DEPENDENC', 'PERTENENCI', 'TIPO', 'AMBITO', 'NOMBRE',
       'CERTIFI', 'DEP_ADM', 'NIVEL', 'COD_COM', 'NOM_COM', 'VIA', 'NUMERO',
       'DIRECCION', 'FONO', 'F_INICIO', 'URGENCIA', 'TIPO_URGE', 'CLAS_SAPU',
       'LATITUD', 'LONGITUD', 'PRESTADOR', 'ESTADO', 'COMPLEJIDA', 'TIPO_ATEN',
       'F_INCR', 'CUT_REG', 'CUT_PROV', 'CUT_COM', 'REGION', 'PROVINCIA',
       'COMUNA', 'geometry'],
      dtype='object')

In [20]:
salud_df.head()

,COD_ANT,COD_VIG,COD_M_ANT,COD_M_NUEV,COD_REG,NOM_REG,COD_DEP,DEPENDENC,PERTENENCI,TIPO,...,COMPLEJIDA,TIPO_ATEN,F_INCR,CUT_REG,CUT_PROV,CUT_COM,REGION,PROVINCIA,COMUNA,geometry
0,02-010,102010.0,None,None,1.0,REGION DE TARAPACA,2.0,SERVICIO DE SALUD TARAPACA,PERTENECIENTE AL SISTEMA NACIONAL DE SERVICIOS...,DIRECCION SERVICIO DE SALUD,...,BAJA COMPLEJIDAD,ATENCION ABIERTA-AMBULATORIA,None,01,011,01101,TARAPACA,IQUIQUE,IQUIQUE,POINT (-70.15232 -20.21512)
1,02-011,102011.0,02-010,102010,1.0,REGION DE TARAPACA,2.0,SERVICIO DE SALUD TARAPACA,PERTENECIENTE AL SISTEMA NACIONAL DE SERVICIOS...,PROGRAMA DE REPARACION Y ATENCION INTEGRAL DE ...,...,BAJA COMPLEJIDAD,ATENCION ABIERTA-AMBULATORIA,None,01,011,01101,TARAPACA,IQUIQUE,IQUIQUE,POINT (-70.15232 -20.21512)
2,02-030,102030.0,None,None,1.0,REGION DE TARAPACA,2.0,SERVICIO DE SALUD TARAPACA,PERTENECIENTE AL SISTEMA NACIONAL DE SERVICIOS...,UNIDAD DE SALUD FUNCIONARIOS,...,BAJA COMPLEJIDAD,ATENCION ABIERTA-AMBULATORIA,None,01,011,01101,TARAPACA,IQUIQUE,IQUIQUE,POINT (-70.15232 -20.21512)
3,02-100,102100.0,None,None,1.0,REGION DE TARAPACA,2.0,SERVICIO DE SALUD TARAPACA,PERTENECIENTE AL SISTEMA NACIONAL DE SERVICIOS...,HOSPITAL,...,ALTA COMPLEJIDAD,ATENCION CERRADA-HOSPITALARIA,None,01,011,01101,TARAPACA,IQUIQUE,IQUIQUE,POINT (-70.13893 -20.21435)
4,02-200,102200.0,None,None,1.0,REGION DE TARAPACA,1.0,SEREMI DE TARAPACA,NO PERTENECIENTE AL SISTEMA NACIONAL DE SERVIC...,CLINICA,...,MEDIANA COMPLEJIDAD,ATENCION CERRADA-HOSPITALARIA,None,01,011,01101,TARAPACA,IQUIQUE,IQUIQUE,POINT (-70.15497 -20.2183)


In [21]:
print(f"Salud DataFrame: {salud_df.shape[0]} registros" ) 

Salud DataFrame: 4613 registros


In [22]:
salud = salud_df[['NOMBRE','TIPO', 'COMUNA', 'geometry']]
salud['COMUNA'] = salud['COMUNA'].str.upper()
salud = salud[salud['COMUNA'].isin(ciudades_list)] #Selecciona las comunas de interés
salud.reset_index(drop=True, inplace=True)
salud.rename(columns={'NOMBRE': 'NOMBRE_ESTABLECIMIENTO', 'TIPO': 'TIPO_ESTABLECIMIENTO'}, inplace=True)
salud

c:\Users\basul\OneDrive\Desktop\u\geoespaciales\geospatial\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,NOMBRE_ESTABLECIMIENTO,TIPO_ESTABLECIMIENTO,COMUNA,geometry
0,ACTIVIDADES GESTIONADAS POR LA DIRECCION DEL S...,DIRECCION SERVICIO DE SALUD,LA SERENA,POINT (-71.24689 -29.90737)
1,PRAIS (S.S COQUIMBO),PROGRAMA DE REPARACION Y ATENCION INTEGRAL DE ...,LA SERENA,POINT (-71.24689 -29.90737)
2,UNIDAD DE SALUD FUNCIONARIO S.S COQUIMBO,UNIDAD DE SALUD FUNCIONARIOS,LA SERENA,POINT (-71.24689 -29.90737)
3,HOSPITAL SAN JUAN DE DIOS (LA SERENA),HOSPITAL,LA SERENA,POINT (-71.24973 -29.90819)
4,CENTRO DE SALUD MUTUAL CCHC LA SERENA,CENTRO DE SALUD PRIVADO,LA SERENA,POINT (-71.25726 -29.91372)
...,...,...,...,...
611,CLINICA DENTAL DR. PATRICIO,CLINICA DENTAL,ARICA,POINT (-70.31793 -18.47889)
612,CLINICA DENTAL INTEGRAL NUEVO AMANECER,CLINICA DENTAL,ARICA,POINT (-70.28443 -18.47265)
613,SALA DE PROCEDIMIENTOS ODONTOLOGICOS DRA. MILI...,CLINICA DENTAL,ARICA,POINT (-70.31184 -18.4826)
614,CLINICA ODONTOLOGICA LUIS PASTEUR E.I.R.L,CLINICA DENTAL,ARICA,POINT (-70.31234 -18.4825)


In [23]:
salud.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 616 entries, 0 to 615
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   NOMBRE_ESTABLECIMIENTO  615 non-null    object  
 1   TIPO_ESTABLECIMIENTO    616 non-null    object  
 2   COMUNA                  616 non-null    object  
 3   geometry                616 non-null    geometry
dtypes: geometry(1), object(3)
memory usage: 19.4+ KB


In [24]:
print(f"Salud DataFrame filtrado: {salud.shape[0]} registros" )

Salud DataFrame filtrado: 616 registros


Un único valor nulo

In [25]:
salud[salud['NOMBRE_ESTABLECIMIENTO'].isnull()]

,NOMBRE_ESTABLECIMIENTO,TIPO_ESTABLECIMIENTO,COMUNA,geometry
208,None,CLINICA DENTAL,VIÑA DEL MAR,POINT (-71.55978 -33.02262)


In [26]:
salud['COMUNA'].unique() #Comprobamos que están las comunas que nos interesan

array(['LA SERENA', 'VALPARAISO', 'VIÑA DEL MAR', 'RANCAGUA',
       'CONCEPCION', 'PUERTO MONTT', 'VALDIVIA', 'ARICA'], dtype=object)

### Establecimientos educacionales

In [27]:
educacion_df.columns

Index(['AGNO', 'RBD', 'DGV_RBD', 'NOM_RBD', 'TIPO_SOST', 'COD_REG_RB',
       'NOM_REG_RB', 'COD_PRO_RB', 'COD_COM_RB', 'NOM_COM_RB', 'COD_DEPROV',
       'NOM_DEPROV', 'TIPO_DEPEN', 'DIRECCION', 'NUMERO', 'REFERENCIA',
       'LATITUD', 'LONGITUD', 'MAT_PARV', 'MAT_BAS_RE', 'MAT_BAS_AD',
       'MAT_ESP', 'MAT_MHC_RE', 'MAT_MHC_AD', 'MAT_MTP_RE', 'MAT_MTP_AD',
       'MAT_TOTAL', 'MAT_HOM_TO', 'MAT_MUJ_TO', 'MAT_SI_TOT', 'CUR_SIM_TO',
       'CUR_COMB_T', 'geometry'],
      dtype='object')

In [28]:
educacion_df.head()

,AGNO,RBD,DGV_RBD,NOM_RBD,TIPO_SOST,COD_REG_RB,NOM_REG_RB,COD_PRO_RB,COD_COM_RB,NOM_COM_RB,...,MAT_MHC_AD,MAT_MTP_RE,MAT_MTP_AD,MAT_TOTAL,MAT_HOM_TO,MAT_MUJ_TO,MAT_SI_TOT,CUR_SIM_TO,CUR_COMB_T,geometry
0,2021.0,8484.0,0.0,ESCUELA PUERTO TORO,1.0,12.0,MAG,122.0,12201.0,CABO DE HORNOS,...,0.0,0.0,0.0,5.0,4.0,1.0,0.0,4.0,0.0,POINT (-67.07622 -55.08317)
1,2021.0,8483.0,2.0,LICEO BICENTENARIO DONALD MC-INTYRE GRIFFITHS,1.0,12.0,MAG,122.0,12201.0,CABO DE HORNOS,...,0.0,0.0,0.0,424.0,212.0,212.0,0.0,15.0,0.0,POINT (-67.60534 -54.9348)
2,2021.0,24341.0,8.0,ESCUELA PAMPA GUANACO,1.0,12.0,MAG,123.0,12303.0,TIMAUKEL,...,0.0,0.0,0.0,5.0,0.0,5.0,0.0,0.0,1.0,POINT (-68.80295 -54.05223)
3,2021.0,8452.0,2.0,ESCUELA DE PUERTO HARRIS,1.0,12.0,MAG,121.0,12101.0,PUNTA ARENAS,...,0.0,0.0,0.0,52.0,29.0,23.0,0.0,0.0,2.0,POINT (-70.45337 -53.835)
4,2021.0,8482.0,4.0,ESCUELA IGNACIO CARRERA PINTO,1.0,12.0,MAG,123.0,12303.0,TIMAUKEL,...,0.0,0.0,0.0,20.0,11.0,9.0,0.0,2.0,2.0,POINT (-69.64666 -53.63903)


In [29]:
print(f"Educacion DataFrame: {educacion_df.shape[0]} registros" ) 

Educacion DataFrame: 11285 registros


In [30]:
educacion = educacion_df[['NOM_RBD','NOM_COM_RB', 'geometry']]
educacion['NOM_COM_RB'] = educacion['NOM_COM_RB'].str.upper()
educacion = educacion[educacion['NOM_COM_RB'].isin(ciudades_list)] #Selecciona las comunas de interés
educacion.reset_index(drop=True, inplace=True)
educacion.rename(columns={'NOM_RBD': 'NOMBRE_ESTABLECIMIENTO', 'NOM_COM_RB': 'COMUNA'}, inplace=True)
educacion

c:\Users\basul\OneDrive\Desktop\u\geoespaciales\geospatial\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,NOMBRE_ESTABLECIMIENTO,COMUNA,geometry
0,ESCUELA RURAL CALETA LA ARENA,PUERTO MONTT,POINT (-72.64285 -41.69405)
1,ESCUELA RURAL HUELMO,PUERTO MONTT,POINT (-73.06259 -41.67305)
2,ESCUELA RURAL CHAICAS,PUERTO MONTT,POINT (-72.66513 -41.62782)
3,ESCUELA RURAL ILQUE,PUERTO MONTT,POINT (-73.08541 -41.62224)
4,ESCUELA RURAL LENCA,PUERTO MONTT,POINT (-72.68983 -41.60532)
...,...,...,...
1163,ESCUELA JORGE ALESSANDRI RODRIGUEZ,ARICA,POINT (-70.29343 -18.4279)
1164,ESCUELA CARLOS CONDELL DE LA HAZA,ARICA,POINT (-70.22585 -18.41711)
1165,LICEO AGR. TEC. PROF. PADRE FRANCISCO NAPOLI,ARICA,POINT (-70.29203 -18.39831)
1166,COLEGIO JUAN PABLO II,ARICA,POINT (-70.31976 -18.39703)


In [31]:
educacion.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1168 entries, 0 to 1167
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   NOMBRE_ESTABLECIMIENTO  1168 non-null   object  
 1   COMUNA                  1168 non-null   object  
 2   geometry                1168 non-null   geometry
dtypes: geometry(1), object(2)
memory usage: 27.5+ KB


In [32]:
print(f"Educacion DataFrame filtrado: {educacion.shape[0]} registros" )

Educacion DataFrame filtrado: 1168 registros


In [33]:
educacion['COMUNA'].unique() #Comprobamos que están las comunas que nos interesan

array(['PUERTO MONTT', 'VALDIVIA', 'CONCEPCIÓN', 'RANCAGUA', 'VALPARAÍSO',
       'VIÑA DEL MAR', 'LA SERENA', 'ARICA'], dtype=object)

##### Establecimientos de educación parvularia

In [34]:
parvularia_df.columns

Index(['AGNO', 'ID_ESTAB', 'ORIGEN', 'NOM_ESTAB', 'COD_REG_ES', 'COD_PRO_ES',
       'COD_COM_ES', 'NOM_COM_ES', 'COD_DEPROV', 'NOM_DEPROV', 'DEPENDENCI',
       'RURAL_ESTA', 'ESTADO_EST', 'SC_MEN_SI', 'SC_MEN_H', 'SC_MEN_M',
       'SC_MAY_H', 'SC_MAY_M', 'MED_MEN_SI', 'MED_MEN_H', 'MED_MEN_M',
       'MED_MAY_SI', 'MED_MAY_H', 'MED_MAY_M', 'NT1_SI', 'NT1_H', 'NT1_M',
       'NT2_SI', 'NT2_H', 'NT2_M', 'N_TOTAL', 'DIRECCION', 'NUMERO',
       'REFERENCIA', 'LATITUD', 'LONGITUD', 'geometry'],
      dtype='object')

In [35]:
parvularia_df.head()

,AGNO,ID_ESTAB,ORIGEN,NOM_ESTAB,COD_REG_ES,COD_PRO_ES,COD_COM_ES,NOM_COM_ES,COD_DEPROV,NOM_DEPROV,...,NT2_SI,NT2_H,NT2_M,N_TOTAL,DIRECCION,NUMERO,REFERENCIA,LATITUD,LONGITUD,geometry
0,2021.0,2.0,1.0,PARVULARIO LAS ESPIGUITAS,15.0,151.0,15101.0,ARICA,151.0,ARICA,...,0.0,41.0,43.0,170.0,RONALD ROSS,2165.0,POB. JUAN NOE,-18.469862,-70.302757,POINT (-70.30276 -18.46986)
1,2021.0,8.0,1.0,COLEGIO INTEGRADO EDUARDO FREI MONTALVA,15.0,151.0,15101.0,ARICA,151.0,ARICA,...,0.0,23.0,9.0,60.0,SAN MARCOS,611.0,ESQUINA GENERAL LAGOS,-18.482491,-70.317581,POINT (-70.31758 -18.48249)
2,2021.0,9.0,1.0,ESCUELA REPUBLICA DE ISRAEL,15.0,151.0,15101.0,ARICA,151.0,ARICA,...,0.0,52.0,55.0,190.0,MAGALLANES,1860.0,ENTRE SIMÓN BOLÍVAR Y BORGOÑO POBLACIÓN MAGIS...,-18.486691,-70.304520,POINT (-70.30452 -18.48669)
3,2021.0,10.0,1.0,ESCUELA REPUBLICA DE FRANCIA,15.0,151.0,15101.0,ARICA,151.0,ARICA,...,0.0,4.0,3.0,15.0,MARIA ILIA DEL PINO,728.0,INTERSECCIÓN DE CALLE SANTA MARÍA CON MARÍA IL...,-18.474369,-70.309036,POINT (-70.30904 -18.47437)
4,2021.0,11.0,1.0,ESC. GRAL. PEDRO LAGOS MARCHANT,15.0,151.0,15101.0,ARICA,151.0,ARICA,...,0.0,8.0,9.0,31.0,AVENIDA R SOTOMAYOR,1639.0,EN EL CERRO LA CRUZ,-18.491011,-70.311577,POINT (-70.31158 -18.49101)


In [36]:
print(f"Parvularia DataFrame: {parvularia_df.shape[0]} registros" ) 

Parvularia DataFrame: 11951 registros


In [37]:
parvularia = parvularia_df[['NOM_ESTAB','NOM_COM_ES', 'geometry']]
parvularia['NOM_COM_ES'] = parvularia['NOM_COM_ES'].str.upper()
parvularia = parvularia[parvularia['NOM_COM_ES'].isin(ciudades_list)] #Selecciona las comunas de interés
parvularia.reset_index(drop=True, inplace=True)
parvularia.rename(columns={'NOM_ESTAB': 'NOMBRE_ESTABLECIMIENTO', 'NOM_COM_ES': 'COMUNA'}, inplace=True)
parvularia

c:\Users\basul\OneDrive\Desktop\u\geoespaciales\geospatial\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,NOMBRE_ESTABLECIMIENTO,COMUNA,geometry
0,PARVULARIO LAS ESPIGUITAS,ARICA,POINT (-70.30276 -18.46986)
1,COLEGIO INTEGRADO EDUARDO FREI MONTALVA,ARICA,POINT (-70.31758 -18.48249)
2,ESCUELA REPUBLICA DE ISRAEL,ARICA,POINT (-70.30452 -18.48669)
3,ESCUELA REPUBLICA DE FRANCIA,ARICA,POINT (-70.30904 -18.47437)
4,ESC. GRAL. PEDRO LAGOS MARCHANT,ARICA,POINT (-70.31158 -18.49101)
...,...,...,...
1269,ANELEY,VALDIVIA,POINT (-73.21675 -39.84486)
1270,ESTRELLITA DE LOS RIOS,VALDIVIA,POINT (-73.25409 -39.84943)
1271,NUBE DE ALGODÓN,VALDIVIA,POINT (-73.2459 -39.8478)
1272,SUEÑOS DE COLORES,VALDIVIA,POINT (-73.20755 -39.83599)


In [38]:
parvularia.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1274 entries, 0 to 1273
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   NOMBRE_ESTABLECIMIENTO  1274 non-null   object  
 1   COMUNA                  1274 non-null   object  
 2   geometry                1274 non-null   geometry
dtypes: geometry(1), object(2)
memory usage: 30.0+ KB


In [39]:
print(f"Parvularia DataFrame filtrado: {parvularia.shape[0]} registros" )

Parvularia DataFrame filtrado: 1274 registros


In [40]:
parvularia['COMUNA'].unique() #Comprobamos que están las comunas que nos interesan

array(['ARICA', 'LA SERENA', 'VALPARAÍSO', 'VIÑA DEL MAR', 'RANCAGUA',
       'CONCEPCIÓN', 'VALDIVIA', 'PUERTO MONTT'], dtype=object)

In [41]:
educa = pd.concat([educacion, parvularia], ignore_index=True) #Unimos los dataframes educacion y parvularia para tener un único dataframe de educación
educa

,NOMBRE_ESTABLECIMIENTO,COMUNA,geometry
0,ESCUELA RURAL CALETA LA ARENA,PUERTO MONTT,POINT (-72.64285 -41.69405)
1,ESCUELA RURAL HUELMO,PUERTO MONTT,POINT (-73.06259 -41.67305)
2,ESCUELA RURAL CHAICAS,PUERTO MONTT,POINT (-72.66513 -41.62782)
3,ESCUELA RURAL ILQUE,PUERTO MONTT,POINT (-73.08541 -41.62224)
4,ESCUELA RURAL LENCA,PUERTO MONTT,POINT (-72.68983 -41.60532)
...,...,...,...
2437,ANELEY,VALDIVIA,POINT (-73.21675 -39.84486)
2438,ESTRELLITA DE LOS RIOS,VALDIVIA,POINT (-73.25409 -39.84943)
2439,NUBE DE ALGODÓN,VALDIVIA,POINT (-73.2459 -39.8478)
2440,SUEÑOS DE COLORES,VALDIVIA,POINT (-73.20755 -39.83599)


# Visualización de datos

# Fuentes
* Geolocalización
    *  Cuerpos de bomberos de chile (2017): [Datos Gob](https://datos.gob.cl/dataset/categoria-geoespacial-instalaciones-y-edificaciones/resource/942164c9-6538-478b-b73c-2672691b36c9)

    * Cuarteles de Carabineros de Chile (2018): [Geoportal](https://www.geoportal.cl/geoportal/catalog/35003/Cuarteles%20de%20Carabineros)

    * Establecimientos de salud (2024): [Geoportal](https://www.geoportal.cl/geoportal/catalog/35499/Establecimientos%20de%20salud%20de%20Chile)

    * Establecimientos de educación escolar (2022): [Geoportal](https://www.geoportal.cl/geoportal/catalog/35408/Establecimientos%20Educaci%C3%B3n%20Escolar)

    * Establecimientos de educación parvularia (2022): [Geoportal](https://www.geoportal.cl/geoportal/catalog/35553/Establecimientos%20Educaci%C3%B3n%20Parvularia)
